# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data_file = "../output_data/cities.csv"
weather_data = pd.read_csv(weather_data_file)
weather_data.head()

,City_ID,City,Lat,Lon,Temperature,Humidity,Cloudiness,Windspeed,Country,Date
0,0,Bluff,-46.60,168.33,50.00,86,84,8.99,NZ,1603151897
1,1,Hobart,-42.88,147.33,59.95,58,20,4.70,AU,1603151856
2,2,Caravelas,-17.71,-39.25,73.54,79,100,5.37,BR,1603152142
3,3,Broadstairs,51.36,1.44,52.84,66,100,8.01,GB,1603152258
4,4,Ushuaia,-54.80,-68.30,41.00,80,75,34.45,AR,1603151860


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
weather_data.dtypes

City_ID          int64
City            object
Lat            float64
Lon            float64
Temperature    float64
Humidity         int64
Cloudiness       int64
Windspeed      float64
Country         object
Date             int64
dtype: object

In [6]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)
locations = weather_data[["Lat", "Lon"]].astype(float)
humidity = weather_data["Humidity"].astype(float)

In [37]:
# Create a Humidity Heatmap layer
fig = gmaps.figure(layout={
        'width': '960px',
        'height': '600px',
        'padding': '3px'
    })

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=True, max_intensity=100)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='600px', padding='3px', width='960px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
